In [31]:
from itertools import chain

from langgraph.prebuilt import ToolNode
from prompt_toolkit import prompt

from config import *
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="qwen-max")

In [32]:
from typing import Annotated, List


@tool
def multiply_by_max(
        a: Annotated[int, "scale factor"],
        b: Annotated[List[int], "list of ints over which to take maximum"],
) -> int:
    """Multiply a by the maximum of b."""
    return a * max(b)


print(multiply_by_max.args_schema.model_json_schema())

{'description': 'Multiply a by the maximum of b.', 'properties': {'a': {'description': 'scale factor', 'title': 'A', 'type': 'integer'}, 'b': {'description': 'list of ints over which to take maximum', 'items': {'type': 'integer'}, 'title': 'B', 'type': 'array'}}, 'required': ['a', 'b'], 'title': 'multiply_by_max', 'type': 'object'}


In [27]:
from langchain_core.prompts import ChatPromptTemplate

In [62]:
from pydantic import BaseModel, Field
from langchain_core.output_parsers import PydanticOutputParser


class Response(BaseModel):
    """Information about a person."""
    name: str = Field(..., description="The name of the function")
    args: dict = Field(..., description="The args of the function.")

In [63]:
parser = PydanticOutputParser(pydantic_object=Response)

In [73]:
prompt_string = """
请根据需求从下列函数中选择合适的函数来完成任务。
函数有：
[\n{functions}\n]

Wrap the output in `json` tags\n{format_instructions}

问题：
{query}
"""
prompt = ChatPromptTemplate.from_messages([("human", prompt_string)]).partial(
    functions=multiply_by_max.args_schema.model_json_schema(),
    format_instructions=parser.get_format_instructions())

In [74]:
prompt.invoke("请将3乘以[1, 2, 3, 4, 5]的最大值。").to_messages()[0].pretty_print()

================================ Human Message =================================


请根据需求从下列函数中选择合适的函数来完成任务。
函数有：
[
{'description': 'Multiply a by the maximum of b.', 'properties': {'a': {'description': 'scale factor', 'title': 'A', 'type': 'integer'}, 'b': {'description': 'list of ints over which to take maximum', 'items': {'type': 'integer'}, 'title': 'B', 'type': 'array'}}, 'required': ['a', 'b'], 'title': 'multiply_by_max', 'type': 'object'}
]

Wrap the output in `json` tags
The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"description": "Information about a person.", "properties": {"nam

In [76]:
chain = prompt | llm
result = chain.invoke("请将3乘以[1, 2, 3, 4, 5]的最大值。")

In [78]:
parser.invoke(result)

Response(name='multiply_by_max', args={'a': 3, 'b': [1, 2, 3, 4, 5]})